**Disclaimer:** Se utilizó ChatGPT para el Reducer de los cuadrados, aunque están basados en el reducer del triangulo que es más simple.

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2c683415aff2767f1046512bfd078c62ee9858b6d57fdd5d206744b3b5560df6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
data = [
    (1,11,2),(1,11,3),(2,11,3),
    (3,11,2),(3,11,4),(4,11,1),
    (4,11,2),(4,11,3),(4,12,5),
    (5,12,1),(5,12,2),(5,12,6)
    ]

**Parte 1**

1.1 Precalentamiento: Triángulos.



**Fase de Map.**

Cada mapper tendra una cierta cantidad de aristas del grafo. Para cada arista (n1, R, n2),
el mapper debe hacer lo siguiente.

1. Hashear los nodos: b1 = h(n1), b2 = h(n2).
2. Como el triangulo que formemos solo puede mapear la variable x a n1 e y a n2, la arista (n1, R, n2)
solo podrıa formar parte de un trıangulo de la forma (n1, n2, a) para un a ∈V . Entonces, generamos b
llaves (b1, b2, 0), . . . , (b1, b2, b −1), y emitimos b mensajes con el contenido de la arista, uno para cada
llave: ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))

Para la primera parte, realizaremos un una función que permita hashear de la forma (key, arista)

**Probamos para Triangulos (X,11,Y), (Y,11,Z), (Z,11,X)**

In [4]:
def key_edge(edge, b):
  map = [] # Lista que contiene los pares key, arista para una determinada edge
  n1, label, n2 = edge # Descomponemos la arista entregada
  for i in range (b): #Iteramos en los valores de mod 2: 0,1
  #agregamos los pares ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))
    map.append(((n1%b, n2%b, i), edge)) #(x,11,y)
    map.append(((i, n1%b, n2%b), edge))  # (y, 11, z)
    map.append(((n2%b, i, n1%b), edge))  # (z, 11, x)
  return map



In [5]:
data_triangles = [edge for edge in data if edge[1] == 11]

rdd = sc.parallelize(data_triangles)
rdd.collect()


[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3)]

In [6]:
map_rdd = rdd.flatMap(lambda x: key_edge(x, 2))
map_rdd.collect()

[((1, 0, 0), (1, 11, 2)),
 ((0, 1, 0), (1, 11, 2)),
 ((0, 0, 1), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 2)),
 ((0, 1, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 3)),
 ((0, 1, 1), (1, 11, 3)),
 ((1, 0, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 0, 1), (2, 11, 3)),
 ((1, 0, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 1), (2, 11, 3)),
 ((1, 1, 0), (2, 11, 3)),
 ((1, 0, 0), (3, 11, 2)),
 ((0, 1, 0), (3, 11, 2)),
 ((0, 0, 1), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 1, 0), (3, 11, 2)),
 ((0, 1, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 4)),
 ((0, 1, 0), (3, 11, 4)),
 ((0, 0, 1), (3, 11, 4)),
 ((1, 0, 1), (3, 11, 4)),
 ((1, 1, 0), (3, 11, 4)),
 ((0, 1, 1), (3, 11, 4)),
 ((0, 1, 0), (4, 11, 1)),
 ((0, 0, 1), (4, 11, 1)),
 ((1, 0, 0), (4, 11, 1)),
 ((0, 1, 1), (4, 11, 1)),
 ((1, 0, 1), (4, 11, 1)),
 ((1, 1, 0), (4, 11, 1)),
 ((0, 0, 0), (4, 11, 2)),
 ((0, 0, 0), (4, 11, 2)),
 ((0, 0, 0),

**Fase de Reduce.**

 Los reducers reciben siempre los mensajes correspondientes a alguna llave (b1, b2, b3).
Pero en el valor de esa llave se encuentran aristas. Todas estas aristas forman un pequeño grafo , y el reducer
emite como respuesta todas las tuplas (n1, n2, n3) correspondientes a los triangulos que detecta en su grafo

In [7]:
group_rdd = map_rdd.groupByKey().mapValues(list)
group_rdd.collect()

[((1, 0, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((0, 1, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((0, 0, 1),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((1, 1, 1), [(1, 11, 3), (1, 11, 3), (1, 11, 3)]),
 ((1, 0, 1),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((1, 1, 0),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 1, 1),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 0, 0), [(4, 11, 2), (4, 11, 2), (4, 11, 2)])]

**Reducer Triangulos**

In [8]:
def reduce_phase(values):
    from collections import defaultdict
    edges_by_node = defaultdict(list)
    for n1, R, n2 in values:
        edges_by_node[(n1, n2)].append(R)

    # Find triangles
    triangles = set()
    nodes = list(edges_by_node.keys())
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            for k in range(j+1, len(nodes)):
                n1, n2 = nodes[i]
                n2_2, n3 = nodes[j]
                n3_2, n1_2 = nodes[k]
                if n2 == n2_2 and n3 == n3_2 and n1 == n1_2:
                    triangles.add((n1, n2, n3))
    return list(triangles)


In [9]:
rdd_triangles = group_rdd.flatMapValues(reduce_phase)
rdd_triangles.collect()

[((1, 0, 0), (2, 3, 4)),
 ((0, 1, 0), (2, 3, 4)),
 ((0, 0, 1), (2, 3, 4)),
 ((1, 0, 1), (1, 3, 4)),
 ((1, 1, 0), (1, 3, 4)),
 ((0, 1, 1), (1, 3, 4))]

**Consulta Cuadrado (x,11,y), (y,11,z), (z,11,w), (w,11,x)**

In [10]:
def key_edge_square(edge, b):
  map = [] # Lista que contiene los pares key, arista para una determinada edge
  n1, label, n2 = edge # Descomponemos la arista entregada
  for i in range (b): #Iteramos en los valores de mod
  #agregamos los pares ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))
    map.append(((n1%b, n2%b, i), edge)) #(x,11,y)
    map.append(((i, n1%b, n2%b), edge))  # (y, 11, z)
    map.append(((n2%b, i, n1%b), edge))  # (z, 11, w)
    map.append(((n1%b, n2%b, i), edge))  # (w, 11, x)
  return map


In [11]:
rdd = sc.parallelize(data_triangles)
rdd.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3)]

In [12]:
map_rdd_square = rdd.flatMap(lambda x: key_edge_square(x, 2))
map_rdd_square.collect()

[((1, 0, 0), (1, 11, 2)),
 ((0, 1, 0), (1, 11, 2)),
 ((0, 0, 1), (1, 11, 2)),
 ((1, 0, 0), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 2)),
 ((0, 1, 1), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 3)),
 ((0, 1, 1), (1, 11, 3)),
 ((1, 0, 1), (1, 11, 3)),
 ((1, 1, 0), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 0, 1), (2, 11, 3)),
 ((1, 0, 0), (2, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 1), (2, 11, 3)),
 ((1, 1, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 0), (3, 11, 2)),
 ((0, 1, 0), (3, 11, 2)),
 ((0, 0, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 1, 0), (3, 11, 2)),
 ((0, 1, 1), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 4)),
 ((0, 1, 0), (3, 11, 4)),
 ((0, 0, 1), (3, 11, 4)),
 ((1, 0, 0), (3, 11, 4)),
 ((1, 0, 1), (3, 11, 4)),
 ((1, 1, 0), (3, 11, 4)),
 ((0, 1, 1),

In [13]:
rdd_group= map_rdd_square.groupByKey().mapValues(list)
rdd_group.collect()


[((1, 0, 0),
  [(1, 11, 2),
   (1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((0, 1, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3),
   (4, 11, 3)]),
 ((0, 0, 1),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 2),
   (4, 11, 3)]),
 ((1, 1, 1), [(1, 11, 3), (1, 11, 3), (1, 11, 3), (1, 11, 3)]),
 ((1, 0, 1),
  [(1, 11, 2),
   (1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((1, 1, 0),
  [(1, 11, 2),
   (1, 11, 3),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 1, 1),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 1),
   (4, 11, 3),
   (4, 11, 3)]),
 ((0, 

In [16]:
def reduce_phase_square_same_label(values):
    from collections import defaultdict

    out_edges = defaultdict(list)

    for n1, R, n2 in values:
        if R == 11:  # We only consider edges with label 11 for this query
            out_edges[n1].append(n2)

    squares = set()
    for n1 in out_edges:
        for n2 in out_edges[n1]:
            if n2 in out_edges:
                for n3 in out_edges[n2]:
                    if n3 in out_edges:
                        for n4 in out_edges[n3]:
                            if n4 in out_edges and n1 in out_edges[n4]:
                                if len(set([n1, n2, n3, n4])) == 4:
                                    square = tuple(sorted([n1, n2, n3, n4]))
                                    squares.add(square)
    return list(squares)





In [17]:
rdd_squares = rdd_group.flatMapValues(reduce_phase_square_same_label)
rdd_squares.collect()

[((1, 0, 0), (1, 2, 3, 4)),
 ((0, 1, 0), (1, 2, 3, 4)),
 ((0, 0, 1), (1, 2, 3, 4)),
 ((1, 0, 1), (1, 2, 3, 4)),
 ((1, 1, 0), (1, 2, 3, 4)),
 ((0, 1, 1), (1, 2, 3, 4))]

**Consulta Cuadrado: (x,11,y), (y,11,z), (z,12,w), (w,12,x)**

In [18]:
rdd = sc.parallelize(data)
rdd.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6)]

In [19]:
rdd_map = rdd.flatMap(lambda x: key_edge_square(x, 2))
rdd_grouped = rdd_map.groupByKey().mapValues(list)
rdd_grouped.collect()

[((1, 0, 0),
  [(1, 11, 2),
   (1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3),
   (4, 12, 5),
   (5, 12, 2),
   (5, 12, 2),
   (5, 12, 6),
   (5, 12, 6)]),
 ((0, 1, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3),
   (4, 11, 3),
   (4, 12, 5),
   (4, 12, 5),
   (5, 12, 2),
   (5, 12, 6)]),
 ((0, 0, 1),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 2),
   (4, 11, 3),
   (4, 12, 5),
   (5, 12, 2),
   (5, 12, 6)]),
 ((1, 1, 1),
  [(1, 11, 3),
   (1, 11, 3),
   (1, 11, 3),
   (1, 11, 3),
   (5, 12, 1),
   (5, 12, 1),
   (5, 12, 1),
   (5, 12, 1)]),
 ((1, 0, 1),
  [(1, 11, 2),
   (1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3),
   (4, 12, 5),
   (5, 12, 1),
   (5, 12, 2),
   (5, 12, 2

In [22]:
def reduce_phase_square_mixed_labels(values):
    from collections import defaultdict

    out_edges = defaultdict(lambda: defaultdict(list))

    for n1, R, n2 in values:
        out_edges[n1][R].append(n2)

    squares = set()
    for n1 in out_edges:
        for n2 in out_edges[n1][11]:  # Edge (n1, 11, n2)
            if n2 in out_edges:
                for n3 in out_edges[n2][11]:  # Edge (n2, 11, n3)
                    if n3 in out_edges:
                        for n4 in out_edges[n3][12]:  # Edge (n3, 12, n4)
                            if n4 in out_edges and n1 in out_edges[n4][12]:  # Edge (n4, 12, n1)
                                if len(set([n1, n2, n3, n4])) == 4:
                                    square = tuple(sorted([n1, n2, n3, n4]))
                                    squares.add(square)
    return list(squares)



In [23]:
rdd_squares_mixed_labels = rdd_grouped.flatMapValues(reduce_phase_square_mixed_labels)
rdd_squares_mixed_labels.collect()

[((1, 0, 0), (2, 3, 4, 5)),
 ((0, 1, 0), (2, 3, 4, 5)),
 ((0, 0, 1), (2, 3, 4, 5)),
 ((1, 0, 1), (1, 3, 4, 5)),
 ((1, 0, 1), (2, 3, 4, 5)),
 ((1, 1, 0), (1, 3, 4, 5)),
 ((1, 1, 0), (2, 3, 4, 5)),
 ((0, 1, 1), (1, 3, 4, 5)),
 ((0, 1, 1), (2, 3, 4, 5))]